In [1]:
from SAC import SAC
from buffer import ReplayBuffer
import gym
from tqdm import tqdm
import torch
from reward_env import RewardEnv
import numpy as np
import torch.nn as nn

In [2]:
env=RewardEnv()

In [3]:
state_dim=env.observation_space.shape[0]
action_dim=env.action_space.shape[0]

In [4]:
replay_buffer=ReplayBuffer(8192)

In [5]:
pb=tqdm(range(100))
for i in pb:
    s=env.reset()
    d=False
    while not d:
        a=env.action_space.sample()
        s_,r,d,_=env.step(a)
        replay_buffer.store(state=s,action=a,log_prob=None,next_state=s_,reward=r,state_value=None,done=d)
        s=s_
    pb.update()

100%|██████████| 100/100 [00:00<00:00, 9806.42it/s]


In [6]:
sac=SAC(30,1,32,1,replay_buffer)

In [7]:
pb=tqdm(range(1000))
for _ in pb:
    sac.update()
    pb.update()

  0%|          | 0/1000 [00:00<?, ?it/s]/home/tools/DR/MyProject/RILE/SAC.py:258: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  q_loss=F.mse_loss(q_value.view(-1),q_hat)
100%|██████████| 1000/1000 [00:02<00:00, 437.56it/s]


In [8]:
index=sac.replay_buffer.sample(2,True)

In [9]:
sac.replay_buffer.state[0]

tensor([-229001.6406, -473013.7500, -299890.8125,  379988.1562, -253203.0312,
         393275.4688, -521069.6875,   31598.9180, -105671.4219,  253317.3281,
        -919356.4375,  210842.7812, -269445.9375, -313047.0000,  886112.9375,
          31573.4316,  117543.6172,  -36410.7461, -939118.4375,  599524.5625,
         619710.1875,  317123.2500,  511041.7500,  603499.1875,  699835.9375,
         986618.5625, -762467.4375,  209534.0156,  276026.9375,  893173.6250])

In [10]:
# class Discriminator(nn.Module):
#     def __init__(self,input_dim,hidden_dim):
#         super(Discriminator,self).__init__()
#         self.discriminator=nn.Sequential(
#             nn.Linear(input_dim,hidden_dim),
#             nn.Tanh(),
#             nn.Linear(hidden_dim,hidden_dim),
#             nn.Tanh(),
#             nn.Linear(hidden_dim,1)
#         )
    
#     def forward(self,x):
#         x=self.discriminator(x)
#         return nn.functional.sigmoid(x)

In [11]:
from Discriminator import Discriminator

In [12]:
D=Discriminator(31,32,64,replay_buffer,replay_buffer)

In [13]:
D.collect_dist(64)

In [14]:
min(D.model.discriminator(D.expert_dist))

tensor([-0.6632], grad_fn=<UnbindBackward0>)

In [15]:
D.update(100)

100%|██████████| 100/100 [00:00<00:00, 786.09it/s]


In [21]:
sa=torch.stack([torch.cat((replay_buffer.state[i],replay_buffer.action[i]),-1) for i in index])

In [24]:
D.model(sa)

tensor([[0.5205],
        [0.4943]], grad_fn=<SigmoidBackward0>)